This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [1]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

In [2]:
from nltk.inference.prover9 import Prover9Exception
import traceback
# looks for a proof from just the premises provided
def try_to_prove_from(statement, premises_list, timeout_in_seconds=60):
    goal = statement
    prover = Prover9Command(goal, assumptions = premises_list, timeout = timeout_in_seconds)
    try:
        prover.prove()
        print(prover.proof())
    except Prover9Exception as e:
        traceback.print_exc()
        #print("Exceeded timeout of", timeout_in_seconds, "sec")

# looks for a proof based on all axioms in event_frame except for those in the exclusion list 
def try_to_prove_without(statement, exclusion_list, timeout_in_seconds=60):
    goal = statement
    prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not ((ax == statement) or ax in exclusion_list)], timeout = timeout_in_seconds)
    try:
        prover.prove()
        print(prover.proof())
    except Prover9Exception as e:
        traceback.print_exc()
        #print("Exceeded timeout of", timeout_in_seconds, "sec")

# looks for a proof based on all axioms in event_frame
def try_to_prove(statement, timeout_in_seconds=60):
    try_to_prove_without(statement,[], timeout_in_seconds)

# looks for a counterexample using Mace4 in which all the premises are true but the statement is false
def try_to_find_counterexample_from(statement, premise_list):
    goal = statement
    mb = MaceCommand(goal, assumptions = premise_list)
    print(mb.build_model())
    print(mb.model(format='cooked'))

# looks for a counterexample using Mace4 in which the statement is false and all the premises in
# event_frame are true with the possible exception of those in exclusion_list
def try_to_find_counterexample_without(statement, exclusion_list = []):
    goal = statement
    mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not ((ax == statement) or ax in exclusion_list)])
    print(mb.build_model())
    print(mb.model(format='cooked'))

# looks for a counterexample using Mace4 in which the statement is false and all the premises in
# event_frame-exclusion_list are true
def try_to_find_counterexample(statement):
    try_to_find_counterexample_without(statement, [])

## Axioms

### Lattice axioms

In [3]:
part_def = read_expr('Partof(x,y) <-> Product(x, y) = x')
proper_part_def = read_expr('Properpartof(x,y) <-> Partof(x,y) & -(x=y)')

sum_id = read_expr('Sum(x,x) = x')
product_id = read_expr('Product(x,x) = x')

sum_comm = read_expr('Sum(x,y) = Sum(y,x)')
product_comm = read_expr('Product(x,y) = Product(y,x)')

sum_assoc = read_expr('Sum(x,Sum(y,z)) = Sum(Sum(x,y),z)')
product_assoc = read_expr('Product(x,Product(y,z)) = Product(Product(x,y),z)')

sum_absorp = read_expr('Sum(x,Product(x,y)) = x')
product_absorp = read_expr('Product(x,Sum(x,y)) = x')

lattice = [part_def, proper_part_def, sum_id, product_id, sum_comm, product_comm, sum_assoc, product_assoc, sum_absorp, product_absorp]

In [4]:
# idempotence of sum and product follow from absorption
try_to_prove_from(product_id, [sum_absorp,  product_absorp])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67289 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 6.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 2.


1 Product(x,x) = x.  [goal].
2 Sum(x,Product(x,y)) = x.  [assumption].
3 Product(x,Sum(x,y)) = x.  [assumption].
4 Product(c1,c1) != c1.  [deny(1)].
6 Product(x,x) = x.  [para(2(a,1),3(a,1,2))].
7 $F.  [resolve(6,a,4,a)].

============================== end of proof ==========================


In [5]:
try_to_prove_from(sum_id, [sum_absorp,product_absorp])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67291 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 6.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 2.


1 Sum(x,x) = x.  [goal].
2 Sum(x,Product(x,y)) = x.  [assumption].
3 Product(x,Sum(x,y)) = x.  [assumption].
4 Sum(c1,c1) != c1.  [deny(1)].
5 Sum(x,x) = x.  [para(3(a,1),2(a,1,2))].
6 $F.  [resolve(5,a,4,a)].

============================== end of proof ==========================


In [6]:
# Proof that parthood is transitive
transitivity_of_parthood = read_expr('Partof(x,y) & Partof(y,z) -> Partof(x,z)')
try_to_prove_from(transitivity_of_parthood, lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67293 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 23.
% Level of proof is 9.
% Maximum clause weight is 11.000.
% Given clauses 34.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
3 Partof(x,y) & Partof(y,z) -> Partof(x,z).  [goal].
7 -Partof(x,y) | Product(x,y) = x.  [clausify(1)].
8 Partof(x,y) | Product(x,y) != x.  [clausify(1)].
11 Sum(x,y) = Sum(y,x).  [assumption].
12 Product(x,y) = Product(y,x).  [assumption].
15 Product(x,Product(y,z)) = Product(Product(x,y),z).  [assumption].
1

In [7]:
def upper_bound(x,P):
    return '(all x1.('+P+'(x1) -> Partof(x1,'+x+')))'

def least_upper_bound(x,P):
    return upper_bound(x,P) + ' & all y1.('+upper_bound('y1','P')+' -> Partof('+x+',y1))'

def lower_bound(x,P):
    return '(all x1.('+P+'(x1) -> Partof('+x+',x1)))'

def greatest_lower_bound(x,P):
    return lower_bound(x,P) + ' & all y1.('+lower_bound('y1','P')+' -> Partof(y1,'+x+'))'

In [8]:
# Proof that sums are unique wherever they exist
unique_sum = read_expr('(('+least_upper_bound('x','P')+' & '+least_upper_bound('y','P')+') -> x=y)')
try_to_prove_from(unique_sum,lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67295 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 5.
% Maximum clause weight is 8.000.
% Given clauses 20.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
3 (all x (P(x) -> Partof(x,y))) & (all z ((all x (P(x) -> Partof(x,z))) -> Partof(y,z))) & (all x (P(x) -> Partof(x,u))) & (all z ((all x (P(x) -> Partof(x,z))) -> Partof(u,z))) -> y = u.  [goal].
7 P(f1(x)) | Partof(c1,x).  [deny(3)].
8 -P(x) | Partof(x,c1).  [deny(3)].
9 -P(x) | Partof(x,c2).  [deny(3)].
10 P

In [9]:
# Proof that products are unique wherever they exist
unique_product = read_expr('(('+greatest_lower_bound('x','P')+' & '+greatest_lower_bound('y','P')+') -> x=y)')
try_to_prove_from(unique_product,lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67297 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 5.
% Maximum clause weight is 8.000.
% Given clauses 20.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
3 (all x (P(x) -> Partof(y,x))) & (all z ((all x (P(x) -> Partof(z,x))) -> Partof(z,y))) & (all x (P(x) -> Partof(u,x))) & (all z ((all x (P(x) -> Partof(z,x))) -> Partof(z,u))) -> y = u.  [goal].
7 P(f1(x)) | Partof(x,c1).  [deny(3)].
8 -P(x) | Partof(c1,x).  [deny(3)].
9 -P(x) | Partof(c2,x).  [deny(3)].
10 P

### Bounded lattice axioms

In [10]:
bot = read_expr('Sum(x, Bot) = x')
top = read_expr('Product(x, Top) = x')

bounded = [bot, top]

bounded_lattice = bounded + lattice

### Distributive lattice axioms

In [11]:
dist_stmt = read_expr('Product(x,Sum(y,z)) = Sum(Product(x,y), Product(x,z))')
dist = [dist_stmt]

dist_lattice = dist + lattice
bounded_dist_lattice = bounded + dist + lattice

In [12]:
#The other distributive lattice statement holds as a theorem
dist2 = read_expr('Sum(x,Product(y,z)) = Product(Sum(x,y), Sum(x,z))')
try_to_prove_from(dist2,dist_lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67299 was started by champollion on standingdesk.home,
Thu Jul 14 17:01:00 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.02 (+ 0.00) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 13.000.
% Given clauses 50.


3 Sum(x,Product(y,z)) = Product(Sum(x,y),Sum(x,z)).  [goal].
11 Product(x,Sum(y,z)) = Sum(Product(x,y),Product(x,z)).  [assumption].
12 Sum(Product(x,y),Product(x,z)) = Product(x,Sum(y,z)).  [copy(11),flip(a)].
16 Product(x,y) = Product(y,x).  [assumption].
17 Sum(x,Sum(y,z)) = Sum(Sum(x,y),z).  [assumption].
18 Sum(Sum(x,y),z) = Sum(x,Sum(y,z)).  [copy(17),fl

### Complete lattice axioms

In [13]:
exist_sum = read_expr('all P.(exists z.'+least_upper_bound('z','P')+')')
exist_product = read_expr('all Q.(exists z.'+greatest_lower_bound('z','Q')+')')

# this only guarantees existence and not uniqueness, 
# but as we have seen above, sums and products are unique wherever they exist

complete = [exist_sum, exist_product]

complete_lattice = complete + lattice
complete_dist_lattice = complete + dist + lattice

### Boolean algebra axioms

In [14]:
# Completeness does not already follow from what we had before
try_to_find_counterexample_from(exist_sum,bounded_dist_lattice)

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

c3 = 0.

f1(0) = 0.
f1(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

- P(0).
- P(1).

  Partof(0,0).
  Partof(0,1).
- Partof(1,0).
  Partof(1,1).

- Properpartof(0,0).
  Properpartof(0,1).
- Properpartof(1,0).
- Properpartof(1,1).



In [22]:
complemented_stmt = read_expr('all x.(exists y. (Product(x, y) = Bot & Sum(x, y) = Top))')
complemented = [complemented_stmt]

#sectional complement
# We call the lattice L sectionally complemented if it has a zero and 
# for all x ≤ y ∈ L, there exists an element z ∈ L satisfying x ∨ z = y and x ∧ z = 0.
sec_compl = read_expr('all x. (all y. (Partof(x,y) -> exists z. (Sum(x, z) = y & Product(x, z) = Bot)))')

boolean_lattice = complemented + dist + lattice
boolean_algebra = boolean_lattice

complete_boolean_lattice = complete + boolean_lattice
complete_boolean_algebra = complete_boolean_lattice

In [23]:
# Every boolean lattice is sectionally complemented
try_to_prove_from(sec_compl, boolean_lattice)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67783 was started by champollion on standingdesk.home,
Thu Jul 14 17:15:14 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 28.
% Level of proof is 5.
% Maximum clause weight is 15.000.
% Given clauses 87.


1 (all x exists y (Product(x,y) = Bot & Sum(x,y) = Top)).  [assumption].
2 Partof(x,y) <-> Product(x,y) = x.  [assumption].
4 (all x all y (Partof(x,y) -> (exists z (Sum(x,z) = y & Product(x,z) = Bot)))).  [goal].
6 -Partof(x,y) | Product(x,y) = x.  [clausify(2)].
9 Partof(c1,c2).  [deny(4)].
14 Product(x,f1(x)) = Bot.  [clausify(1)].
15 Sum(x,f1(x)) = Top.  [

In [24]:
# Every boolean algebra is sectionally complemented
try_to_prove_from(sec_compl, boolean_algebra)

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 67786 was started by champollion on standingdesk.home,
Thu Jul 14 17:15:16 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.02 (+ 0.00) seconds.
% Length of proof is 28.
% Level of proof is 5.
% Maximum clause weight is 15.000.
% Given clauses 87.


1 (all x exists y (Product(x,y) = Bot & Sum(x,y) = Top)).  [assumption].
2 Partof(x,y) <-> Product(x,y) = x.  [assumption].
4 (all x all y (Partof(x,y) -> (exists z (Sum(x,z) = y & Product(x,z) = Bot)))).  [goal].
6 -Partof(x,y) | Product(x,y) = x.  [clausify(2)].
9 Partof(c1,c2).  [deny(4)].
14 Product(x,f1(x)) = Bot.  [clausify(1)].
15 Sum(x,f1(x)) = Top.  [

In [25]:
# This succeeds, which suggests that either I've made a coding error or
# not every bounded distributive lattice is sectionally complemented.
try_to_find_counterexample_from(sec_compl, bounded_dist_lattice)

True
% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 2.

c2 = 1.

Product(0,0) = 0.
Product(0,1) = 0.
Product(0,2) = 0.
Product(1,0) = 0.
Product(1,1) = 1.
Product(1,2) = 2.
Product(2,0) = 0.
Product(2,1) = 2.
Product(2,2) = 2.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(0,2) = 2.
Sum(1,0) = 1.
Sum(1,1) = 1.
Sum(1,2) = 1.
Sum(2,0) = 2.
Sum(2,1) = 1.
Sum(2,2) = 2.

  Partof(0,0).
  Partof(0,1).
  Partof(0,2).
- Partof(1,0).
  Partof(1,1).
- Partof(1,2).
- Partof(2,0).
  Partof(2,1).
  Partof(2,2).

- Properpartof(0,0).
  Properpartof(0,1).
  Properpartof(0,2).
- Properpartof(1,0).
- Properpartof(1,1).
- Properpartof(1,2).
- Properpartof(2,0).
  Properpartof(2,1).
- Properpartof(2,2).



### Exclusion axioms

In [26]:
cumulativity = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(Sum(x1, x2), Sum(y1, y2))')
symmetry = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [29]:
#define conflict:
confl_def = read_expr('Confl(x, y) <-> exists z1.(Partof(z1,x) & exists z2.(Partof(z2,y) & Excl(z1, z2)))')

#define possibilities:
#int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
#ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(Product(y, z) = y & int_poss(z) & -(confl(z,x))))')

poss_def = read_expr('Poss(x) <-> -Confl(x, x)')

#define world:
#world_def = read_expr('world(x) <-> (int_poss(x) & all y.((Product(x, y) = y) -> -int_poss(y)))')

world_def = read_expr('World(x) <-> (Poss(x) & all y.(Properpartof(x,y) -> -Poss(y)))')

#Harmony
#harmony = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

harmony = read_expr('all x.(all y. ((World(x) & -Confl(x, y)) -> Poss(y)))')

#Nirvana
nirvana = read_expr('all x.(-Confl(Bot, x))')

#Rashōmon
#rashomon = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(Product(x,y)))')
rashomon = read_expr('all x y.((Poss(x) & Poss(y) & -Confl(x, y)) -> Poss(Sum(x,y)))')


#Cosmopolitanism
#cosmopol = read_expr('all x.(int_poss(x) -> exists y.(world(y) & Product(x, y) = y))')

cosmopol = read_expr('all x.(Poss(x) -> exists y.(World(y) & Partof(x,y)))')



#actual world
actual_world = read_expr('World(actual_world)')

#define actual:
actual_def = read_expr('Actual(x) <-> Partof(x, actual_world)')

exclusion_axioms = [cumulativity, symmetry, harmony, nirvana, rashomon, cosmopol, actual_world]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

definitions = [confl_def, poss_def, world_def, actual_def]

event_frame = complete_boolean_algebra + exclusion_axioms + definitions

## Independence of axioms

In [30]:
# assumptions tested for exclusion: actual_world, cosmopol, complemented, cumulativity, dist, harmony, rashomon
# If we don't assume Nirvana, Mace4 finds a counterexample that shows symmetry is an axiom.
try_to_find_counterexample_without(symmetry,[nirvana])

True
% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

actual_world = 0.

c1 = 0.

c2 = 1.

f5(0) = 1.
f5(1) = 0.

f6(0) = 0.
f6(1) = 0.

f9(0) = 0.
f9(1) = 0.

Product(0,0) = 0.
Product(0,1) = 0.
Product(1,0) = 0.
Product(1,1) = 1.

Sum(0,0) = 0.
Sum(0,1) = 1.
Sum(1,0) = 1.
Sum(1,1) = 1.

f1(0,0) = 1.
f1(0,1) = 1.
f1(1,0) = 1.
f1(1,1) = 1.

f3(0,0) = 0.
f3(0,1) = 0.
f3(1,0) = 0.
f3(1,1) = 0.

f7(0,0) = 0.
f7(0,1) = 0.
f7(1,0) = 0.
f7(1,1) = 0.

f8(0,0) = 0.
f8(0,1) = 1.
f8(1,0) = 0.
f8(1,1) = 1.

f2(0,0,0) = 0.
f2(0,0,1) = 0.
f2(0,1,0) = 0.
f2(0,1,1) = 0.
f2(1,0,0) = 1.
f2(1,0,1) = 0.
f2(1,1,0) = 1.
f2(1,1,1) = 0.

f4(0,0,0) = 0.
f4(0,0,1) = 0.
f4(0,1,0) = 0.
f4(0,1,1) = 0.
f4(1,0,0) = 0.
f4(1,0,1) = 0.
f4(1,1,0) = 0.
f4(1,1,1) = 0.

  Actual(0).
- Actual(1).

  P(0).
  P(1).

  Poss(0).
- Poss(1).

- Q(0).
- Q(1).

  World(0).
- World(1).

- Confl(0,0).
  Confl(0,1).
- Confl(1,0).
  Confl(1,1).

- Excl(0,0).
  Excl(0,1).
- Excl(1,0).
- Excl(1,1).

  Partof(0

In [ ]:
# If we do, it doesn't find that counterexample. So symmetry might be a theorem.
#try_to_find_counterexample_without(symmetry,[])  # times out

In [ ]:
# Even though symmetry might be a theorem, we can't know for sure because the search for a proof also times out.
# try_to_prove_without(symmetry,[]) # times out

In [ ]:
# assumptions tested for exclusion: 
# actual_world, cosmopol, complemented, cumulativity, dist, harmony, nirvana, rashomon

# If we don't assume Nirvana, Mace4 finds a counterexample that shows cumulativity is an axiom.
# In that counterexample there are just two elements: Bot and Top. Bot is the actual world, Bot and Top exclude each other but not themselves.
#try_to_find_counterexample_without(cumulativity,[nirvana])
# assumptions tested for exclusion: actual_world, cosmopol, complemented, dist, harmony, rashomon

#There is also a counterexample if we include Nirvana but not Complemented. 
# Size 3. Bot=0, Top=1, actual_world = 2. No worlds other than actual_world. 
# Top is impossible. Actual_world and Top exclude each other. Top conflicts with itself but doesn't exclude itself.
#try_to_find_counterexample_without(cumulativity,[complemented])

# If we insist on Nirvana and Complemented, Mace4 doesn't find a counterexample:
# try_to_find_counterexample_without(cumulativity,[]) # times out

In [ ]:
# But if we try to prove cumulativity, it also times out:
# try_to_prove(cumulativity) # times out
# So we don't know if cumulativity is a theorem. If it isn't, the counterexamples are likely infinitely sized.

In [ ]:
try_to_find_counterexample(cumulativity) # cumulativity is an axiom

In [ ]:
try_to_find_counterexample(symmetry) # symmetry is an axiom

In [ ]:
try_to_find_counterexample(harmony) # succeeds as expected

In [ ]:
try_to_prove(harmony,1) # times out at 60 as expected because a countermodel can be found

In [ ]:
try_to_find_counterexample(harmony) # finds countermodel -- as expected

In [ ]:
try_to_find_counterexample(cosmopol) # fails as expected because countermodels are infinite

In [ ]:
try_to_prove(cosmopol) # times out after at least 60 sec -- as expected

In [ ]:
#Nirvana is not derivable from other axioms
try_to_prove(nirvana,1) # times out as expected after at least 60 sec

In [ ]:
try_to_find_counterexample(nirvana) # finds counterex as expected

In [ ]:
#Rashōmon is not derivable from other possibility axioms (times out at 60 sec)
try_to_prove(rashomon,1)

In [ ]:
try_to_find_counterexample(rashomon) # finds counterexample as expected

In [ ]:
#Cosmopolitanism is not derivable from other possibility axioms (counterexample found)
# So this times out as expected
try_to_prove(cosmopol,1)

In [ ]:
#Cosmopolitanism is not derivable from other possibility axioms -- but there are no *finite* countermodels
# so this returns False as expected
try_to_find_counterexample(cosmopol)

## Theorems

### Plenitude

In [ ]:
plenitude = read_expr('all x.((exists y.(-Confl(x, y) & World(y))) <-> Poss(x))')

In [ ]:
# Plenitude follows from Harmony, Cosmopol, and Symmetry. If we remove any of these, the proof search times out.
try_to_prove_without(plenitude,[nirvana, rashomon, cumulativity,complemented, dist,actual_world]) 
#not needed: nirvana, rashomon, cumulativity, complemented, dist, actual_world
# apparently needed: harmony, cosmopol, symmetry

### Distributivity of actuality

In [33]:
#Since Actual is defined as being part of the actual world, distributivity of actuality follows just from 
# transitivity of parthood, which holds in every lattice
distofact = read_expr('all x.((exists y. (Actual(y) & Partof(x,y))) -> Actual(x))')
try_to_prove_from(distofact, lattice + [actual_def])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 68204 was started by champollion on standingdesk.home,
Thu Jul 14 17:29:45 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 29.
% Level of proof is 11.
% Maximum clause weight is 11.000.
% Given clauses 37.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
3 Actual(x) <-> Partof(x,actual_world).  [assumption].
4 (all x ((exists y (Actual(y) & Partof(x,y))) -> Actual(x))).  [goal].
8 -Partof(x,y) | Product(x,y) = x.  [clausify(1)].
9 Partof(x,y) | Product(x,y) != x.  [clausify(1)].
12 Sum(x,y) = Sum(y,x).  [assumption].
13 Product(x,y) = Product(y,x).  [assumpti

In [ ]:
#Find counterexample for distributivity of actuality - fails as expected
try_to_find_counterexample(distofact)

### Cumulativity of actuality

In [35]:
#Cumulativity of actuality also follows from the basic properties of lattices
cumofact = read_expr('all x y.((Actual(x) & Actual(y)) -> Actual(Sum(x,y)))')
try_to_prove_from(cumofact, lattice + [actual_def])

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 68273 was started by champollion on standingdesk.home,
Thu Jul 14 17:30:12 2022
The command was "prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 29.
% Level of proof is 8.
% Maximum clause weight is 11.000.
% Given clauses 127.


1 Partof(x,y) <-> Product(x,y) = x.  [assumption].
3 Actual(x) <-> Partof(x,actual_world).  [assumption].
4 (all x all y (Actual(x) & Actual(y) -> Actual(Sum(x,y)))).  [goal].
8 -Partof(x,y) | Product(x,y) = x.  [clausify(1)].
9 Partof(x,y) | Product(x,y) != x.  [clausify(1)].
12 Sum(x,y) = Sum(y,x).  [assumption].
13 Product(x,y) = Product(y,x).  [assumption

In [ ]:
#Find counterexample for cumulativity of actuality - fails as expected
try_to_find_counterexample(cumofact)

### Manichaeism

In [ ]:
#Manichaeism depends for its proof on Harmony and Rashomon but on surprisingly little else
manichaeism = read_expr('all x.((World(x) -> all y.(Partof(y,x) | Confl(x, y))))')
try_to_prove_without(manichaeism,[actual_world,cosmopol,complemented,cumulativity,dist,nirvana,symmetry])

### Appropriateness of conjunction

In [ ]:
#Appropriateness of conjunction depends on very little: it holds in any lattice
appr_conj = read_expr(r'((exists x. (P(x) & Actual(x))) & (exists y. (Q(y) & Actual(y)))) <-> exists y exists z.(Actual(y) & Actual(z) & P(y) & Q(z) & Actual(Sum(y, z)))')
try_to_prove_from(appr_conj,lattice+[actual_def])

### Appropriateness of disjunction

In [ ]:
#Similarly, appropriateness of disjunction holds in any lattice
appr_disj = read_expr(r'(exists x. (Actual(x) & (P(x) | Q(x)))) <-> ((exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) & Q(y))))')
try_to_prove_from(appr_disj,lattice+[actual_def])

### No Gaps - Lucas' version

In [ ]:
#e precludes Q iff ∃S. (e = ⊔S) ∧ (∀x∈S ∃y∈Q ∃y1⊑y, x⊥y1) ∧ (∀y∈Q ∃x∈S ∃y1⊑y, x⊥y1)

def precludes(e, Q):
    return 'exists S. ('+least_upper_bound(e,'S')+' & (all x. (S(x) -> exists y. (Q(y) & exists y1. (Partof(y1, y) & Excl(x,y1))))) & (all y. (Q(y) -> exists x (S(y) & exists y1. (Partof(y1,y) & Excl(x,y1))))))'

#example = 'exists P. ((all x1.(P(x1) -> Partof(x1,e))) & all y1.((all x1.(P(x1) -> Partof(x1,y1))) -> Partof(e,y1))) & (all x. (P(x) -> exists y. (Q(y) & exists y1. (Partof(y1, y) & Excl(x,y1))))) & (all y. (Q(y) -> exists x (P(y) & exists y1. (Partof(y1,y) & Excl(x,y1)))))'

#example_string = precludes('e','Q')
#print(example_string)
#example_expr = read_expr(example_string)
#example_expr

#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x
#def negate(x, P):
#    result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
#              & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
#    return result

In [ ]:
#check no gaps
no_gaps = read_expr('(exists x. (Actual(x) & P(x))) | (exists y. (Actual(y) &'+precludes('y', 'P')+'))')
print(no_gaps)

In [ ]:
try_to_prove(no_gaps,70)

In [ ]:
try_to_find_counterexample(no_gaps)

### No Gluts - Lucas' version

In [ ]:
#check no gluts
no_gluts = read_expr('(-exists x. Actual(x) & P(x)) | (-exists y. Actual(y) &'+precludes('y', 'P')+')')
print(no_gluts)

In [ ]:
try_to_prove(no_gluts,70)

In [ ]:
try_to_find_counterexample(no_gluts)
# Finds a counterexample: Two-element model. Bot = 0, Top = 1. The actual world is Top. 
# Both Bot and Top are actual. No exclusions hold.
# Predicate P holds of both Bottom and Top. Predicate Q holds of neither.
# P holds of something actual. No Gluts would say that no actual y precludes P. 
# Since Bot and Top are actual this would mean that nothing precludes P. 
# Because of the last conjunct of the preclusion definition, indeed nothing will preclude P
# because P is nonempty and the exclusion relation is empty. 

### No Gaps - Zhuoye's version

In [ ]:
#define negating a set
#x negates P iff there is a function h such that x = ⊔{h(xi) ∣ xi ∈ P} and for all events 
# x1 ∈ P, h(x1) excludes some part of x1.
# 
# x negates P iff every x1 in P has a part y that excludes some part x2 of x and
# every part x3 of x

# def negate(x, P):
#     result = ('(all x1. (P(x1) -> (exists x2. Partof(x2,x) & exists y. Partof(y,x1) & Excl(x2, y)))) 
#               & (all x3. ((Partof(x3,x) & (-exists x4. (Partof(x4,x) & Partof(x4,x3)))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(Partof(x8,x) & Partof(x9,x) & x8!=x9 & Excl(x6, x8) & Excl(x7, x9))))')
#     return result


#define exclusionary negation: an event x exclusionarily negates P iff
#there is a set of events Q such that for each event x in P, part of x is excluded by some event y in Q
#and each event y in Q excludes some part of an event x in P
#and x is the sum of Q.
def exclusionary_negate(x, P):
    result = '(exists Q. ((all x1. (' + P + '(x1) -> (exists x2. Partof(x2, x1) & (exists y1. (Q(y1) & Excl(y1, x2))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
              '& (all y3. (Q(y3) -> Partof(y3,'+x+'))) & (all y5. ((all y4.(Q(y4) -> Partof(y4, y5))) -> Partof('+x+', y5)))))'
    return result

#define preclude a set: an event x precludes P iff there exists a Q such that
#for any x1 in P, there is y1 in Q such that y1 excludes part of x1
#for any y2 in Q, there is x3 in P such that y2 excludes part of x3
#for any distinct y3, y4 in Q, there are distinct x5, x6 such that y3 excludes part of x5, y4 excludes part of x6
#x is the sum of Q
def preclude(x, P):
    result = '(exists Q. ((all x1.('+P+'(x1) -> (exists y1. (Q(y1) & (exists x2. (Partof(x2, x1) & Excl(y1, x2)))))))'\
             '& (all y2. (Q(y2) -> (exists x3. ('+P+'(x3) & (exists x4. (Partof(x4, x3) & Excl(y2, x4)))))))'\
             '& (all y3. (all y4. ((Q(y3) & Q(y4) & y3 != y4) -> (exists x5. (exists x6. ('+P+'(x5) & '+P+'(x6) & exists x7. (exists x8. (Partof(x7, x5) & (Partof(x8, x6) & Excl(y3, x7) & Excl(y4, x8))))))))))'\
             '& ((all y5. (Q(y5) -> Partof(y5, '+x+'))) & (all y6. ((all y7. (Q(y7) -> Partof(y7, y6))) -> Partof('+x+', y6))))))'
    return result

In [ ]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

In [ ]:
#check no gaps with preclusion-based negation
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
prover = Prover9Command(goal, assumptions = event_frame, timeout=1000)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. ((exists z. P(z)) -> (all x. (World(x) -> (exists y. (Partof(y, x) & (P(y)|'+preclude('y', 'P')+'))))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

### No Gluts - Zhuoye's version

In [ ]:
#check no gluts with exclusionary negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+exclusionary_negate('z', 'P')+')))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))

In [ ]:
#check no gluts with preclusion-based negation
goal = read_expr(r'all P. (-(exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+')))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr(r'all P. (-exists x. (World(x) & (exists y. (Partof(y, x) & P(y))) & (exists z. (Partof(z, x) &'+preclude('z', 'P')+'))))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()
print(mb.model(format = 'cooked'))